<a href="https://colab.research.google.com/github/LucasColas/Movie-review/blob/main/IMDB_movie_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## prerequisites

In [ ]:
!pip install --upgrade datasets fsspec s3fs huggingface_hub


INFO: pip is looking at multiple versions of s3fs to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.4/515.4 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 78.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.33.1
    Uninstalling huggingface-hub-0.33.1:
      Successfully uninstalled huggingface-hub-0.33.1
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-

In [ ]:
!pip install --upgrade transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 51.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.0
    Uninstalling transformers-4.53.0:
      Successfully uninstalled transformers-4.53.0


## libraries

In [ ]:
from collections import Counter
from datasets import load_dataset, concatenate_datasets
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


## dataset

In [ ]:


# Load IMDB: train/test split with 25k examples each.
# Labels : 0 (negative), 1 (positive)
imdb = load_dataset("imdb")
train_ds, test_ds = imdb["train"], imdb["test"]


In [ ]:
train_ds

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [ ]:
display(train_ds[0])

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [ ]:
# Count labels in the training set
train_label_counts = Counter(train_ds["label"])
print("Label counts in the training dataset:", train_label_counts)

# Count labels in the testing set
test_label_counts = Counter(test_ds["label"])
print("Label counts in the testing dataset:", test_label_counts)

Label counts in the training dataset: Counter({0: 12500, 1: 12500})
Label counts in the testing dataset: Counter({0: 12500, 1: 12500})


In [ ]:
# merge half the test set with the training set.

test_ds_label_0 = test_ds.filter(lambda x: x['label'] == 0)
test_ds_label_1 = test_ds.filter(lambda x: x['label'] == 1)


num_to_move_0 = len(test_ds_label_0) // 2
num_to_move_1 = len(test_ds_label_1) // 2

# examples to move from test to train
move_ds_label_0 = test_ds_label_0.select(range(num_to_move_0))
move_ds_label_1 = test_ds_label_1.select(range(num_to_move_1))

# new training set
new_train_ds = concatenate_datasets([train_ds, move_ds_label_0, move_ds_label_1])

# the remaining examples are for the new test set
remain_ds_label_0 = test_ds_label_0.select(range(num_to_move_0, len(test_ds_label_0)))
remain_ds_label_1 = test_ds_label_1.select(range(num_to_move_1, len(test_ds_label_1)))
new_test_ds     = concatenate_datasets([remain_ds_label_0, remain_ds_label_1])

new_train_ds = new_train_ds.shuffle(seed=42)


train_ds, test_ds = new_train_ds, new_test_ds

# 9. Verify label distributions
print("Label counts in new TRAIN set:", Counter(train_ds["label"]))
print("Label counts in new TEST  set:", Counter(test_ds["label"]))

Label counts in new TRAIN set: Counter({1: 18750, 0: 18750})
Label counts in new TEST  set: Counter({0: 6250, 1: 6250})


## model

In [ ]:


model_name = "prajjwal1/bert-tiny"
tokenizer  = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["text"],
                     padding="max_length",
                     truncation=True,
                     max_length=256)

train_tok = train_ds.map(tokenize, batched=True)
test_tok  = test_ds.map(tokenize, batched=True)

# Set format for PyTorch
train_tok.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_tok.set_format ( "torch", columns=["input_ids", "attention_mask", "label"])


Map:   0%|          | 0/37500 [00:00<?, ? examples/s]

Map:   0%|          | 0/12500 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# Load pretrained model with a classification head
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Define training hyperparameters
args = TrainingArguments(
    output_dir="imdb-transformer",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds  = pred.predictions.argmax(-1)
    acc    = accuracy_score(labels, preds)
    p, r, f, _ = precision_recall_fscore_support(labels, preds, average="binary")
    return {"accuracy": acc, "precision": p, "recall": r, "f1": f}

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_tok,
    eval_dataset=test_tok,
    compute_metrics=compute_metrics,
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## training

In [ ]:

trainer.train()
results = trainer.evaluate()
print(results)


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.575100,0.446763,0.798800,0.803116,0.791680,0.797357
2,0.409300,0.379284,0.835200,0.803447,0.887520,0.843394
3,0.362100,0.349035,0.850000,0.849720,0.850400,0.850060
4,0.334800,0.338216,0.853760,0.861156,0.843520,0.852247
5,0.318400,0.332857,0.858560,0.863681,0.851520,0.857557
6,0.306200,0.329025,0.861840,0.852753,0.874720,0.863597
7,0.293800,0.327034,0.861280,0.850948,0.876000,0.863292
8,0.286200,0.328337,0.861040,0.872421,0.845760,0.858884
9,0.279600,0.326676,0.864080,0.854384,0.877760,0.865914
10,0.275500,0.325666,0.863520,0.856001,0.874080,0.864946


{'eval_loss': 0.32566601037979126, 'eval_accuracy': 0.86352, 'eval_precision': 0.8560012535255406, 'eval_recall': 0.87408, 'eval_f1': 0.8649461684610513, 'eval_runtime': 104.8092, 'eval_samples_per_second': 119.264, 'eval_steps_per_second': 3.731, 'epoch': 10.0}


In [ ]:
# Alternative : use fp16
args = TrainingArguments(
    output_dir="imdb-transformer",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
    fp16_opt_level="O1",
    gradient_accumulation_steps=2,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_tok,
    eval_dataset=test_tok,
    compute_metrics=compute_metrics,
)

trainer.train()
results = trainer.evaluate()
print(results)

## Inference

In [ ]:
from transformers import pipeline
nlp = pipeline("text-classification", model=trainer.model, tokenizer=tokenizer, return_all_scores=True)
example = "This movie was not as good as I expected."
print(nlp(example))

Device set to use cpu


[[{'label': 'LABEL_0', 'score': 0.3659629225730896}, {'label': 'LABEL_1', 'score': 0.6340370178222656}]]


/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
